In [ ]:
   def u_stat_indep_matrix(self, data_X, data_Y):
    n = data_X.size(dim=0)

    Phi = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
    Psi = torch.matmul(data_Y, torch.transpose(data_Y, 0, 1))
    
    Phi_tilde = Phi.fill_diagonal_(0.0)
        Psi_tilde = Psi.fill_diagonal_(0.0)
        n_four = n * (n-1) * (n-2) * (n-3)
        one = torch.ones(n, 1).to(device)
        oneT = torch.transpose(one, 0, 1)

        PhiPsi = torch.matmul(Phi, Psi)
        trPhiPsi = torch.trace(PhiPsi)
        onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

        onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
        onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
        onePhioneonePsione = torch.matmul(onePhione, onePsione)

        Un = (
          4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
        - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
        + 4 * (n-3)*(n-2) * trPhiPsi
        )
        
        return(Un/n_four)




In [ ]:
def u_stat_indep_matrix_efficient(self, data_X, data_Y):

   #scalars
   n = data_X.size(dim = 0)
   n_four = n * (n-1) * (n-2) * (n-3)
   
   #matrices
   Phi = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
   Psi = torch.matmul(data_Y, torch.transpose(data_Y, 0, 1))
   Phi_tilde = Phi.fill_diagonal_(0.0)
   Psi_tilde = Psi.fill_diagonal_(0.0)

        one = torch.ones(n, 1).to(self.cuda_device)
        oneT = torch.transpose(one, 0, 1)

        PhiPsi = torch.matmul(Phi, Psi)
        trPhiPsi = torch.trace(PhiPsi)
        onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

        onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
        onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
        onePhioneonePsione = torch.matmul(onePhione, onePsione)

 

   Un = 4 * onePhioneonePsione
   - ( 16 + (8 * (n-3)) ) * onePhiPsiOne
   + ( 8 + (8 * (n-3))  + (4 * (n-3)*(n-2)) ) * trPhiPsi     
        
        return(Un/n_four)

In [ ]:
    def u_stat_indep_original(self, data_X, data_Y):
        n = data_X.size(dim = 0)
        print(f"number of calculation = {2*scipy.special.comb(n,4) }")
        n_four = n * (n-1) * (n-2) * (n-3)
        U_statistic = 0
        for i in range(n):
            set_j = set(range(n)) - {i}
            for j in set_j:
                set_k = set_j - {j}
                for k in set_k:
                    set_r = set_k - {k}
                    for r in set_r:
                        comb = [i,j,k,r]
                        U_statistic = U_statistic + (
                            self.kernel_indep(data_X[comb,]) * self.kernel_indep(data_Y[comb,])
                        )/n_four
        return(U_statistic)

In [ ]:
  #original form
   #Un = (
   #  4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
   # - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
   # + 4 * (n-3)*(n-2) * trPhiPsi
   # )

   # six terms
   #Un = 4 * onePhioneonePsione
   #- 16 * onePhiPsiOne
   #+ 8  * trPhiPsi
   #- 8 * (n-3) * onePhiPsiOne
   #+ 8 * (n-3) * trPhiPsi
   #+ 4 * (n-3)*(n-2) * trPhiPsi

   # aggregate
   # Un = 4 * onePhioneonePsione
   # - ( 16 + (8 * (n-3)) ) * onePhiPsiOne
   # + ( 8 + (8 * (n-3))  + (4 * (n-3)*(n-2)) ) * trPhiPsi